# GPT 3.5 Turbo - Information Extraction

In [1]:
import os
import openai
import numpy as np 
import pandas as pd
import time 
import pdfkit
import re
import copy
from tqdm import tqdm
from itertools import product
openai.organization = os.environ.get("OPENAI_ORGANIZATION")
openai.api_key = os.environ.get("OPENAI_API_KEY")

C:\Users\lucid\AppData\Roaming\Python\Python39\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\lucid\AppData\Roaming\Python\Python39\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
C:\Users\lucid\AppData\Roaming\Python\Python39\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
pd.set_option('display.max_rows', 500)

In [3]:
openai.Model.retrieve('gpt-3.5-turbo')

<Model model id=gpt-3.5-turbo at 0x1d76cca5220> JSON: {
  "created": 1677610602,
  "id": "gpt-3.5-turbo",
  "object": "model",
  "owned_by": "openai",
  "parent": null,
  "permission": [
    {
      "allow_create_engine": false,
      "allow_fine_tuning": false,
      "allow_logprobs": true,
      "allow_sampling": true,
      "allow_search_indices": false,
      "allow_view": true,
      "created": 1683753011,
      "group": null,
      "id": "modelperm-YO9wdQnaovI4GD1HLV59M0AV",
      "is_blocking": false,
      "object": "model_permission",
      "organization": "*"
    }
  ],
  "root": "gpt-3.5-turbo"
}

In [4]:
edmundsreviews = pd.read_csv('Edmunds_CustomerReviews.csv', encoding = "utf-16")
edmundsreviews.drop('Unnamed: 0', inplace = True, axis = 1)
edmundsreviews.head()

youtubereviews = pd.read_csv('YoutubeComments_40_InfoRich.csv')
youtubereviews.drop('Unnamed: 0', axis = 1, inplace = True)

carsreviews = pd.read_csv('CarsCustomerReviews.csv')
carsreviews.drop('Unnamed: 0', axis = 1, inplace = True)

kbbreviews = pd.read_csv('KBBCustomerReviews.csv')
kbbreviews.drop('Unnamed: 0', axis = 1, inplace = True)

In [9]:
#ChatGPT API
def trash_to_treasure(df):
    dflist = []
    #For debug purposes - API issues, rate limits etc. 
    error = []
    error_index = []
    counter = 0
    comments = df["Review"].tolist()
    for comment in tqdm(comments):
        segment = df.iloc[counter]['Segment']
        model = df.iloc[counter]['Model']
        make = df.iloc[counter]['Make']
        year = df.iloc[counter]['Year']
        try:
            content = openai.ChatCompletion.create(
            model = 'gpt-3.5-turbo',
            messages=[
        {"role": "system", "content": "You are a helpful assistant that helps answer questions and perform the specific task "
         + "being asked."},
        {"role": "user", "content":" We are attempting to understand what vehicle features customers truly care about and what "\
         + "the pain points are based on customer comments. Note that the comments are related to "\
         + "automobile reviews hence pay more attention to certain words like Blind Spot Detection, Braking, Steering, "\
         + "Acceleration and other Car related features if they exist in the comments. Also make sure to focus only on the "\
         + "present vehicle being talked about and not on thoughts on similar vehicles owned at a prior time. With that in mind, "\
         + "what are the positives and the negative takeaways the customers had based on the following comment? Did the customer "\
         + "have any specific wishes? If so, list those out as part of a separate wishlist. Note that if the customer had a wish "\
         + "or a requested feature or doesn't list any relevant vehicle features, do not consider that as a positive or a negative. "\
         + "For each comment, formulate the answer with Positives, Negatives and Wishlist categories with short keywords per bullet. "\
         + "Do not number the bullet points. Summarize each bullet into specific keywords relating to vehicle "\
         + "features and make it a priority keep it as short as possible, around 3 - 4 words per bullet. Prefer concise words representing vehicle features over longer phrases. Do not add anything"\
         + " else to the response. Do not add an overall or summary section Wishlist. The following comment is in regards to the vehicle "+ df.iloc[counter]['Make'] + ' ' + df.iloc[counter]['Model'] +".  Comment : "+comment}
         ],
        temperature = 1,
        max_tokens= 300,
        frequency_penalty = 0,
        presence_penalty = 0,
        #n=3
            ).get("choices")[0]['message']['content'].split('\n')
    
    #Data Cleaning, Data Formatting into a consistent response programatically - Can only control output to an extent with 
    #prompt engineering. 
    #Blank Space Removal, Replacing Bullets with Blanks, Capturing different occurences of Categories to slice them into 
    #appropriate data formats such as list of tuples to enable creation of the final dataframe. 
            if '' in content:
                content.remove('')
            ret = [x.replace('- ', '') for x in content]
            posstring = ''
            for x in ret:
                find = re.findall(r'^Positives:.*', x)
                if find != []:
                    posstring = posstring + find[0]
                    break
            
            negstring = ''
            for x in ret:
                find = re.findall(r'^Negatives:.*', x)
                if find != []:
                    negstring = negstring + find[0]
                    break
            
            wishstring = ''
            for x in ret:
                find = re.findall(r'^Wishlist:.*', x)
                if find != []:
                    wishstring = wishstring + find[0]
                    break
        
            #segment = df.iloc[counter]['Segment']
            #model = df.iloc[counter]['Model']
            #make = df.iloc[counter]['Make']
            #year = df.iloc[counter]['Year']
            try:
                p = ret.index(posstring)
                n = ret.index(negstring)
                w = ret.index(wishstring)
                positives = ', '.join(ret[p+1:n])
                negatives = ', '.join(ret[n+1:w])
                wishlist = ', '.join(ret[w+1:])
            except:
            #This exception comes into play in case there's no relevant information in any of the categories or in context to 
            #Nissan vehicles where ChatGPT outputs a warning. This is done to maintain indexing and subsequent metadata rather
            #than skipping the iteration entirely. 
                positives = ''
                negatives = ''
                wishlist = ''
                print('Trigger')
                #dflist.append((counter, segment, model, make, year, positives, negatives, wishlist))
                #continue
    
            dflist.append((counter, segment, model, make, year, positives, negatives, wishlist))
            counter = counter + 1
            time.sleep(0.5)
    
    #This exception is used in case of any rate errors or API connectivity issues so that continuation is possible from the 
    #remaining comments. 
        except Exception as e:
            error.append(e)
            error_index.append(counter)
            positives = ''
            negatives = ''
            wishlist = ''
            print('Trigger_2')
            dflist.append((counter, segment, model, make, year, positives, negatives, wishlist))
            counter = counter + 1
            continue
            #return (dflist, error, error_index) 
    return (dflist, error, error_index)

In [ ]:
#Depending on the run, you would get problematic indexes. These can be found at t1[2]. Rerun the function with trash_to_treasure(t1[2]) and create dataframe, export it and then concatenate it. 
#This process has been repeated multiple times to get the final output for the first pass. The Original comments aren't needed 
#after this point. This method was used to generate multiple files which have then been concatenated and processed. 
#t1 = trash_to_treasure(youtubereviews)
#t1 = trash_to_treasure(edmundsreviews)
#t1 = trash_to_treasure(kbbreviews)
t1 = trash_to_treasure(carsreviews.iloc[t1[2]])

In [ ]:
#Due to long processing times, the process was done in batches.
#It can be processed as follows.
#if t1[2] is not empty, re-run the above function with just those indexes 
#Eg : trash_to_treasure(kbbreviews.iloc[t1[2]])
df = pd.DataFrame(t1[0], columns = ["Counter", "Segment", "Model", "Make", "Year", "Positives", "Negatives", "Wishlist"])
df.drop('Counter', axis = 1, inplace = True)
df = df.drop_duplicates()
df.to_csv('CarsExtractPFinal.csv')
#df.to_csv('filename.csv') or df.to_excel('filename.xlsx')

In [ ]:
#Due to long processing times, the process was done in batches. 
df = pd.read_excel('EdmundsextractP1.xlsx')
df2 = pd.read_excel('EdmundsextractP2.xlsx')
df3 = pd.read_excel('EdmundsextractP3.xlsx')
df4 = pd.read_excel('EdmundsextractP4.xlsx')
df5 = pd.read_excel('EdmundsextractP5.xlsx')
df6 = pd.read_excel('EdmundsextractP6.xlsx')
df7 = pd.read_excel('EdmundsextractP2_5.xlsx')
df8 = pd.read_excel('EdmundsextractPLast.xlsx')
df9 = pd.read_excel('EdmundsextractPErrorIndexes.xlsx')
df10 = pd.read_excel('EdmundsextractErrorLast.xlsx')
df_final = pd.concat([df, df2, df7, df3, df4, df5, df6, df8, df9, df10])
df_final = df_final.reset_index()
df_final.drop(['index', 'Unnamed: 0', 'Counter'], axis = 1, inplace = True)
#Don't drop duplicates
#df_final = df_final.drop_duplicates()
df_final['Positives'] = df_final['Positives'].fillna('None mentioned')
df_final['Negatives'] = df_final['Negatives'].fillna('None mentioned')
df_final['Wishlist'] = df_final['Wishlist'].fillna('None mentioned')
df_final.to_csv('EdmundsPass1.csv')

df = pd.read_csv('YoutubeExtractP1.csv')
df2 = pd.read_csv('YoutubeExtractP2.csv')
df3 = pd.read_csv('YoutubeExtractP3.csv')
df4 = pd.read_csv('YoutubeExtractP4.csv')
df5 = pd.read_csv('YoutubeExtractP5.csv')
df7 = pd.read_csv('YoutubeExtractP6.csv')
df6 = pd.read_csv('YoutubeExtractPLast.csv')
df_final = pd.concat([df, df2, df3, df4, df5, df7, df6])
df_final = df_final.reset_index()
df_final.drop(['index', 'Unnamed: 0', 'Counter'], axis = 1, inplace = True)
#Don't drop duplicates
#df_final = df_final.drop_duplicates()
df_final['Positives'] = df_final['Positives'].fillna('None mentioned')
df_final['Negatives'] = df_final['Negatives'].fillna('None mentioned')
df_final['Wishlist'] = df_final['Wishlist'].fillna('None mentioned')
df_final.to_csv('YoutubePass1.csv')

df = pd.read_csv('KBBExtractP1.csv')
df2 = pd.read_csv('KBBExtractP1_5.csv')
df3 = pd.read_csv('KBBExtractP2.csv')
df4 = pd.read_csv('KBBExtractP2_5.csv')
df_final = pd.concat([df, df2, df3, df4])
df_final = df_final.reset_index()
df_final.drop(['index', 'Unnamed: 0'], axis = 1, inplace = True)
#Don't drop duplicates
#df_final = df_final.drop_duplicates()
df_final['Positives'] = df_final['Positives'].fillna('None mentioned')
df_final['Negatives'] = df_final['Negatives'].fillna('None mentioned')
df_final['Wishlist'] = df_final['Wishlist'].fillna('None mentioned')
df_final.to_csv('KBBPass1.csv')

df = pd.read_csv('CarsExtractP1.csv')
df2 = pd.read_csv('CarsExtractP1_5.csv')
df3 = pd.read_csv('CarsExtractP1_75.csv')
df4 = pd.read_csv('CarsExtractP2.csv')
df5 = pd.read_csv('CarsExtractP2_25.csv')
df6 = pd.read_csv('CarsExtractP2_5.csv')
df7 = pd.read_csv('CarsExtractP2_75.csv')
df8 = pd.read_csv('CarsExtractP3.csv')
df9 = pd.read_csv('CarsExtractP4.csv')
df10 = pd.read_csv('CarsExtractPFinal.csv')
df_final = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9, df10])
df_final = df_final.reset_index()
df_final.drop(['index', 'Unnamed: 0'], axis = 1, inplace = True)
#Don't drop duplicates
#df_final = df_final.drop_duplicates()
df_final['Positives'] = df_final['Positives'].fillna('None mentioned')
df_final['Negatives'] = df_final['Negatives'].fillna('None mentioned')
df_final['Wishlist'] = df_final['Wishlist'].fillna('None mentioned')
df_final.to_csv('CarsPass1.csv')

In [ ]:
dfed = pd.read_csv('EdmundsPass1.csv')
dfyt = pd.read_csv('YoutubePass1.csv')
dfkbb = pd.read_csv('KBBPass1.csv')
dfcars = pd.read_csv('CarsPass1.csv')
dfPass1 = pd.concat([dfed, dfyt, dfkbb, dfcars])
dfPass1 = dfPass1.reset_index()
dfPass1.drop(['index', 'Unnamed: 0'], axis = 1, inplace = True)
dfPass1 = dfPass1.dropna()
dfPass1.to_csv('Pass1_Complete.csv')

In [ ]:
dfPass1

In [ ]:
df_final = pd.read_csv('Pass1_Complete.csv')
df_final.drop('Unnamed: 0', inplace = True, axis = 1)
dfp = pd.DataFrame(df_final[["Segment", "Model", "Make", "Year", "Positives", "Negatives", "Wishlist"]])
dfpe = pd.DataFrame(dfp.assign(Positives = dfp['Positives'].str.split(',')).explode('Positives'))
dfpe = pd.DataFrame(dfpe.assign(Positives = dfpe['Positives'].str.split(' and ')).explode('Positives'))
dfpe = pd.DataFrame(dfpe.assign(Negatives = dfpe['Negatives'].str.split(',')).explode('Negatives'))
dfpe = pd.DataFrame(dfpe.assign(Negatives = dfpe['Negatives'].str.split(' and ')).explode('Negatives'))
dfpe = pd.DataFrame(dfpe.assign(Wishlist = dfpe['Wishlist'].str.split(',')).explode('Wishlist'))
dfpe = pd.DataFrame(dfpe.assign(Wishlist = dfpe['Wishlist'].str.split(' and ')).explode('Wishlist'))
dfP = dfpe[["Segment", "Make", "Model", "Year", "Positives"]].reset_index()
dfP.drop('index', axis = 1, inplace = True)
#dfP = dfP.dropna()
dfP = dfP.rename(columns={'Positives': 'Features'})
dfN = dfpe[["Segment", "Make", "Model", "Year", "Negatives"]].reset_index()
dfN.drop('index', axis = 1, inplace = True)
#dfN = dfN.dropna()
dfN = dfN.rename(columns={'Negatives': 'Features'})
dfW = dfpe[["Segment", "Make",  "Model", "Year", "Wishlist"]].reset_index()
dfW.drop('index', axis = 1,  inplace = True)
#dfW = dfW.dropna()
dfW = dfW.rename(columns={'Wishlist': 'Features'})

In [ ]:
#Do not drop duplicates in the Original Row Wise Exploded Positives for Frequency Analysis. 
#Note : This code is not used as processing is slightly different for Similarity/Frequency Analysis
#dfEdmundsP.to_csv('ExplodedPositives.csv')
#dfEdmundsN.to_csv('ExplodedNegatives.csv')
#dfEdmundsW.to_csv('ExplodedWishlist.csv')

#Remove duplicates for creating the masterlist as we don't need the duplicates for this process. 
dfP.drop_duplicates(subset = 'Features').to_csv('ExplodedPositives.csv')
dfN.drop_duplicates(subset = 'Features').to_csv('ExplodedNegatives.csv')
dfW.drop_duplicates(subset = 'Features').to_csv('ExplodedWishlist.csv')